In [1]:
import csv
import uuid
import os


In [2]:
def initialize_csv_files():
    if not os.path.exists("users.csv"):
        with open("users.csv", "w", newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['username', 'password'])

    if not os.path.exists("flights.csv"):
        with open("flights.csv", "w", newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['flight_id', 'origin', 'destination', 'date', 'time', 'seats', 'available_seats', 'price'])
            writer.writerow([1, 'New York', 'London', '2025-07-20', '14:00', 100, 100, 500])
            writer.writerow([2, 'Paris', 'Berlin', '2025-07-21', '09:30', 50, 50, 200])
            writer.writerow([3, 'Tokyo', 'Sydney', '2025-07-22', '22:15', 80, 80, 750])

    if not os.path.exists("bookings.csv"):
        with open("bookings.csv", "w", newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['booking_id', 'username', 'flight_id', 'seats_booked', 'total_price'])

initialize_csv_files()
print(" CSV files ready.")


 CSV files ready.


In [3]:
def register():
    username = input("Choose a username: ")
    password = input("Choose a password: ")

    with open("users.csv", "a", newline='') as f:
        writer = csv.writer(f)
        if os.path.getsize("users.csv") == 0:
            writer.writerow(['username', 'password'])  # Write header
        writer.writerow([username, password])
        print(" Registered successfully")


In [4]:
register()

Choose a username:  rohit
Choose a password:  rohit


 Registered successfully


In [5]:
def login():
    username = input("Enter your username: ")
    password = input("Enter your password: ")

    with open("users.csv", "r") as f:
        reader = csv.DictReader(f)
        for row in reader:
            if row['username'] == username and row['password'] == password:
                print(f"Login successful! Welcome, {username}")
                return username
                
    print("Invalid username or password.")
    return None


In [6]:
login()


Enter your username:  kenil
Enter your password:  keni


Login successful! Welcome, kenil


'kenil'

In [7]:
def view_flights():
    print("\n Available Flights ")
    with open("flights.csv", "r") as f:
        reader = csv.DictReader(f)
        flights = list(reader)

        if not flights:
            print("No flights available.")
            return

        for flight in flights:
            print(f"""
Flight ID:        {flight['flight_id']}
From:             {flight['origin']}
To:               {flight['destination']}
Date:             {flight['date']}
Time:             {flight['time']}
Price:            ${flight['price']}
Seats Available:  {flight['available_seats']}
     """)




def add_flight():
    flight_id = input("Flight ID: ")
    origin = input("Origin: ")
    destination = input("Destination: ")
    date = input("Date (YYYY-MM-DD): ")
    time = input("Time (HH:MM): ")
    seats = int(input("Total Seats: "))
    price = float(input("Ticket Price: "))

    with open("flights.csv", "a", newline='') as f:
        writer = csv.writer(f)
        writer.writerow([flight_id, origin, destination, date, time, seats, seats, price])
        print("Flight added successfully.")



def delete_flight():
    flight_id_to_delete = input("Enter the Flight ID to delete: ")
    updated_flights = []

    with open("flights.csv", "r") as f:
        reader = csv.DictReader(f)
        for row in reader:
            if row['flight_id'] != flight_id_to_delete:
                updated_flights.append(row)

    if len(updated_flights) == 0:
        print("No flights remain after deletion.")
        return

    # Rewrite the file with the updated list
    with open("flights.csv", "w", newline='') as f:
        writer = csv.DictWriter(f, fieldnames=updated_flights[0].keys())
        writer.writeheader()
        writer.writerows(updated_flights)
        print(f"Flight ID {flight_id_to_delete} deleted.")




In [8]:
view_flights()


 Available Flights 

Flight ID:        1
From:             New York
To:               London
Date:             2025-07-20
Time:             14:00
Price:            $500
Seats Available:  97
     

Flight ID:        2
From:             Paris
To:               Berlin
Date:             2025-07-21
Time:             09:30
Price:            $200
Seats Available:  40
     

Flight ID:        3
From:             Tokyo
To:               Sydney
Date:             2025-07-22
Time:             22:15
Price:            $750
Seats Available:  69
     

Flight ID:        4
From:             Mumbai
To:               Dubai
Date:             2025-01-18
Time:             9:30
Price:            $275.0
Seats Available:  72
     

Flight ID:        5
From:             Dubai
To:               Barcelona
Date:             2025-05-15
Time:             4:40
Price:            $370.0
Seats Available:  90
     

Flight ID:        6
From:             Mumbai
To:               Canada
Date:             2025-04-24
Time: 

In [9]:
def book_flight(username):
    view_flights()
    flight_id = input("Enter the Flight ID to book: ")
    seats_requested = int(input("How many seats do you want to book? "))

    flights = []
    flight_found = False

    with open("flights.csv", "r") as f:
        reader = csv.DictReader(f)
        for row in reader:
            if row['flight_id'] == flight_id:
                flight_found = True
                available_seats = int(row['available_seats'])
                price_per_seat = float(row['price'])

                if seats_requested > available_seats:
                    print(" Not enough seats available.")
                    return
                else:
                    row['available_seats'] = str(available_seats - seats_requested)
                    total_price = seats_requested * price_per_seat
                    booking_id = str(uuid.uuid4())[:8]

                    with open("bookings.csv", "a", newline='') as bf:
                        writer = csv.writer(bf)
                        writer.writerow([booking_id, username, flight_id, seats_requested, total_price])

                    print(f" Booking Confirmed! Booking ID: {booking_id}")
                    print(f"Total Price: ${total_price}")
            flights.append(row)

    if not flight_found:
        print(" Flight ID not found.")
        return

    with open("flights.csv", "w", newline='') as f:
        writer = csv.DictWriter(f, fieldnames=flights[0].keys())
        writer.writeheader()
        writer.writerows(flights)


In [10]:
 username = login()


Enter your username:  kenil
Enter your password:  keni


Login successful! Welcome, kenil


In [11]:
def search_flights():
    origin = input("Enter origin city: ").strip().lower()
    destination = input("Enter destination city: ").strip().lower()

    found = False
    with open("flights.csv", "r") as f:
        reader = csv.DictReader(f)
        print("\n Available Flights:")
        for row in reader:
            if (row['origin'].strip().lower() == origin and
                row['destination'].strip().lower() == destination):
                found = True
                print(f"ID: {row['flight_id']} | {row['origin']} -> {row['destination']} on {row['date']} at {row['time']} | ${row['price']} | {row['available_seats']} seats left")

    if not found:
        print("No matching flights.")


In [12]:
search_flights()

Enter origin city:  mumbai
Enter destination city:  dubai



 Available Flights:
ID: 4 | Mumbai -> Dubai on 2025-01-18 at 9:30 | $275.0 | 72 seats left


In [26]:
book_flight(username)


 Available Flights 

Flight ID:        1
From:             New York
To:               London
Date:             2025-07-20
Time:             14:00
Price:            $500
Seats Available:  97
     

Flight ID:        2
From:             Paris
To:               Berlin
Date:             2025-07-21
Time:             09:30
Price:            $200
Seats Available:  40
     

Flight ID:        3
From:             Tokyo
To:               Sydney
Date:             2025-07-22
Time:             22:15
Price:            $750
Seats Available:  69
     

Flight ID:        4
From:             Mumbai
To:               Dubai
Date:             2025-01-18
Time:             9:30
Price:            $275.0
Seats Available:  70
     

Flight ID:        5
From:             Dubai
To:               Barcelona
Date:             2025-05-15
Time:             4:40
Price:            $370.0
Seats Available:  90
     

Flight ID:        6
From:             Mumbai
To:               Canada
Date:             2025-04-24
Time: 

Enter the Flight ID to book:  14
How many seats do you want to book?  2


 Booking Confirmed! Booking ID: 2a43eecf
Total Price: $680.0


In [27]:
def view_my_bookings(username):
    print(f"\n--- {username}'s Bookings ---")
    bookings_found = False

    with open("bookings.csv", "r") as bf:
        reader = csv.DictReader(bf)
        for row in reader:
            if row['username'] == username:
                bookings_found = True
                print("\n Your Ticket:")
                print(f"""-------------
Booking ID:    {row['booking_id']}
Flight ID:     {row['flight_id']}
Seats Booked:  {row['seats_booked']}
Total Price:   ${row['total_price']}
----------------------------""")

    if not bookings_found:
        print(" No bookings found.")


In [28]:

with open("bookings.csv", "r") as f:
    rows = list(csv.reader(f))


with open("bookings.csv", "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['booking_id', 'username', 'flight_id', 'seats_booked', 'total_price'])  # correct header
    writer.writerows(rows)  


In [29]:
view_my_bookings(username)


--- kenil's Bookings ---

 Your Ticket:
-------------
Booking ID:    2a43eecf
Flight ID:     14
Seats Booked:  2
Total Price:   $680.0
----------------------------


In [30]:
def cancel_booking(username):
    # Read all bookings
    with open("bookings.csv", "r") as f:
        bookings = list(csv.DictReader(f))

    # Filter user's bookings
    user_bookings = [b for b in bookings if b.get('username') == username]
    if not user_bookings:
        print(" No bookings to cancel.")
        return

    # Display user's bookings
    print("\n Your Bookings:")
    for i, b in enumerate(user_bookings, 1):
        print(f"{i}. Flight ID: {b.get('flight_id', 'N/A')}")

    try:
        choice = input("Enter number to cancel (or 'q' to quit): ")
        if choice.lower() == 'q':
            return
        idx = int(choice) - 1
        cancel = user_bookings[idx]
    except (ValueError, IndexError):
        print(" Invalid selection.")
        return

    # Remove booking from list
    bookings = [b for b in bookings if not (
        b.get('username') == cancel.get('username') and
        b.get('flight_id') == cancel.get('flight_id') and
        b.get('date') == cancel.get('date')
    )]

    # Rewrite bookings.csv
    with open("bookings.csv", "w", newline='') as f:
     writer = csv.DictWriter(f, fieldnames=["booking_id", "username", "flight_id", "seats_booked", "total_price"])
     writer.writeheader()
     writer.writerows(bookings)


    # Restore seat in flights.csv
    with open("flights.csv", "r") as f:
        flights = list(csv.DictReader(f))

    for flight in flights:
        if flight.get("flight_id") == cancel.get("flight_id"):
            flight["available_seats"] = str(int(flight["available_seats"]) + 1)

    with open("flights.csv", "w", newline='') as f:
        writer = csv.DictWriter(f, fieldnames=flights[0].keys())
        writer.writeheader()
        writer.writerows(flights)

    print(" Booking canceled and seat restored.")


In [31]:
cancel_booking(username)


 Your Bookings:
1. Flight ID: 14


Enter number to cancel (or 'q' to quit):  q


In [32]:
def view_all_bookings():
    with open("bookings.csv", "r") as f:
        reader = csv.DictReader(f)
        print("\n📋 All Bookings:")
        for row in reader:
            print(f"{row['booking_id']} | {row['username']} | Flight {row['flight_id']} | Seats: {row['seats_booked']} | ${row['total_price']}")


In [33]:
view_all_bookings()


📋 All Bookings:
booking_id | username | Flight flight_id | Seats: seats_booked | $total_price
username | flight_id | Flight None | Seats: None | $None
2a43eecf | kenil | Flight 14 | Seats: 2 | $680.0


In [34]:
def user_dashboard(username):
    while True:
        print(f"\n Welcome, {username}")
        print("1. View Available Flights")
        print("2. Book a Flight")
        print("3. View My Bookings")
        print("4. Cancel Booking")
        print("5. Log out")

        choice = input("Choose an option: ")
        if choice == '1':
            view_flights()
        elif choice == '2':
            book_flight(username)
        elif choice == '3':
            view_my_bookings(username)
        elif choice == '4':
            cancel_booking(username)
        elif choice == '5':
            print(" Logged out.")
            break
        else:
            print(" Invalid choice.")


In [35]:
user_dashboard(username)


 Welcome, kenil
1. View Available Flights
2. Book a Flight
3. View My Bookings
4. Cancel Booking
5. Log out


Choose an option:  3



--- kenil's Bookings ---

 Your Ticket:
-------------
Booking ID:    2a43eecf
Flight ID:     14
Seats Booked:  2
Total Price:   $680.0
----------------------------

 Welcome, kenil
1. View Available Flights
2. Book a Flight
3. View My Bookings
4. Cancel Booking
5. Log out


Choose an option:  5


 Logged out.


In [36]:
def main():
    while True:
        print("\n Airline Reservation System ")
        print("1. Register")
        print("2. Login")
        print("3. Exit")

        choice = input("Choose an option: ")
        if choice == '1':
            register()
        elif choice == '2':
            user = login()
            if user:
                user_dashboard(user)
        elif choice == '3':
            print(" Goodbye!")
            break
        else:
            print(" Invalid choice.")




In [39]:
##### main()


 Airline Reservation System 
1. Register
2. Login
3. Exit


Choose an option:  1
Choose a username:  Dishant
Choose a password:  dishu


 Registered successfully

 Airline Reservation System 
1. Register
2. Login
3. Exit


Choose an option:  2
Enter your username:  Dishant
Enter your password:  dishu


Login successful! Welcome, Dishant

 Welcome, Dishant
1. View Available Flights
2. Book a Flight
3. View My Bookings
4. Cancel Booking
5. Log out


...


--- Dishant's Bookings ---

 Your Ticket:
-------------
Booking ID:    cc7821a8
Flight ID:     4
Seats Booked:  2
Total Price:   $550.0
----------------------------

 Welcome, Dishant
1. View Available Flights
2. Book a Flight
3. View My Bookings
4. Cancel Booking
5. Log out


Choose an option:  5


 Logged out.

 Airline Reservation System 
1. Register
2. Login
3. Exit


Choose an option:  3


 Goodbye!
